In [1]:
import numpy as np
import pandas as pd
from timeit import default_timer as timer
from numba import jit, njit, vectorize, cfunc, prange
from scipy import sparse
from scipy.sparse.linalg import spsolve
import networkx as nx
from scipy.sparse.csgraph import reverse_cuthill_mckee
import math

In [2]:
#####################
# friction functions#
#####################
@njit(parallel=True)
def f_tot(epsD_vec,Re_vec):
    N=len(Re_vec)
    f_vec=np.zeros(N)
    fp_vec=np.zeros(N)
    for i in prange(N):
        Re=Re_vec[i]
        epsD=epsD_vec[i]
        if Re<=2000:
            #############
            #laminar flow
            f=64/Re 
            fp=-64/(Re*Re)
        elif Re>=4000:
            #################
            ###turbulent flow
            #swamee jain#####
            y1=5.74/np.power(Re,0.9)
            y2=epsD/3.7+y1
            c1=-2/math.log(10)
            y3=c1*math.log(y2)
            f=1/(y3*y3)

            y3=c1*math.log(y2)
            fp=(2*0.9*c1)*y1*f/(y2*y3*Re)
        else:
            ######################
            # transition flow####
            #dunlop interpolation
            ######################
            aa=-1.5634601348517065795
            #aa=-2*0.9*2/math.log(10)
            ab=0.00328895476345399058690
            #ab=5.74/np.power(4000,0.9)
            c2=-2/math.log(10)
            #c2=-0.8685889638065035

            ac=aa*ab

            y2=epsD/3.7+ab
            y3=c2*math.log(y2)

            fa=1/(y3*y3)
            fb=(2+ac/(y2*y3))*fa

            x1=7*fa-fb
            x2=0.128-17*fa+2.5*fb
            x3=-0.128+13*fa-(fb+fb)
            x4=0.032-3*fa+0.5*fb

            R=Re/2000

            f=x1+R*(x2+R*(x3+R*x4))
            fp=x2+R*(2*x3+3*R*x4)
            fp=fp/2000
        f_vec[i]=f
        fp_vec[i]=fp
    return [f_vec,fp_vec]








@njit
def smoke1(F, b, i, j,Np):
    for k in range(Np):
        F[i[k]]+=b[k]
        F[j[k]]-=b[k]

#####################
#  other function   #
#####################
def file_read(name_,name_n):
    node = pd.read_csv(name_[name_n]+'\\junction.txt',sep='\t')
    pipe = pd.read_csv(name_[name_n]+'\\pipe.txt',sep='\t')
    reservoir =   pd.read_csv(name_[name_n]+'\\reservoir.txt',sep='\t')
    return [node, pipe, reservoir]

In [3]:
np.show_config()

blas_mkl_info:
  NOT AVAILABLE
blis_info:
  NOT AVAILABLE
openblas_info:
    library_dirs = ['D:\\a\\1\\s\\numpy\\build\\openblas_info']
    libraries = ['openblas_info']
    language = f77
    define_macros = [('HAVE_CBLAS', None)]
blas_opt_info:
    library_dirs = ['D:\\a\\1\\s\\numpy\\build\\openblas_info']
    libraries = ['openblas_info']
    language = f77
    define_macros = [('HAVE_CBLAS', None)]
lapack_mkl_info:
  NOT AVAILABLE
openblas_lapack_info:
    library_dirs = ['D:\\a\\1\\s\\numpy\\build\\openblas_lapack_info']
    libraries = ['openblas_lapack_info']
    language = f77
    define_macros = [('HAVE_CBLAS', None)]
lapack_opt_info:
    library_dirs = ['D:\\a\\1\\s\\numpy\\build\\openblas_lapack_info']
    libraries = ['openblas_lapack_info']
    language = f77
    define_macros = [('HAVE_CBLAS', None)]


In [4]:
def f_lam(epsD,Re):
    f=64/Re 
    fp=-64/(Re*Re)
    return [f,fp]

def f_turb(epsD,Re):
    #################
    ###turbulent flow
    #swamee jain#####
    y1=5.74/np.power(Re,0.9)
    y2=epsD/3.7+y1
    c1=-2/np.log(10)
    y3=c1*np.log(y2)
    f=1/(y3*y3)

    y3=c1*np.log(y2)
    fp=(2*0.9*c1)*y1*f/(y2*y3*Re)
    
    return [f,fp]

def f_trans(epsD,Re):
    oo=np.ones(len(Re))
    ######################
    # transition flow####
    #dunlop interpolation
    ######################
    aa=-1.5634601348517065795
    #aa=-2*0.9*2/np.log(10)
    ab=0.00328895476345399058690
    #ab=5.74/np.power(4000,0.9)
    c2=-2/np.log(10)
    #c2=-0.8685889638065035

    ac=aa*ab

    y2=epsD/3.7+ab*oo
    y3=c2*np.log(y2)

    fa=1/(y3*y3)
    fb=(2*oo+ac/(y2*y3))*fa

    x1=7*fa-fb
    x2=0.128*oo-17*fa+2.5*fb
    x3=-0.128*oo+13*fa-(fb+fb)
    x4=0.032*oo-3*fa+0.5*fb

    R=Re/2000

    f=x1+R*(x2+R*(x3+R*x4))
    fp=x2+R*(2*x3+3*R*x4)
    fp=fp/2000
    
    return [f,fp]

In [5]:
## file read ##
name_n=3


In [6]:
## file read ##
###############
start = timer()
###############
name_=['example4','example5','bin','rome'];
demand_multip=np.array([1,1,0.45,1])

a=file_read(name_,name_n)

node=a[0];
pipe=a[1];
reservoir=a[2];

print('file read, time=',timer() - start,'s')
print('\n')




## initialize##
###############
start = timer()
###############

node_id=np.array(node.ID)-1
elev=np.array(node.Elev) #m
demand=np.array(node.Demand/1000)# %m^3/s
demand=demand*demand_multip[name_n]
pipe_id=np.array(pipe.ID)-1
pipe_node=np.array([np.array(pipe.Node1)-1 , np.array(pipe.Node2)-1])
len_=np.array(pipe.Length) #m
diam=np.array(pipe.Diameter/1000) #m
rough=np.array(pipe.Roughness/1000) #m
rsrv_=[np.array(reservoir.ID) ,np.array(reservoir.Head)]   # head(m)
rsrv_1=rsrv_[1]

Nr=len(rsrv_)
Np=len(pipe_id)
Nj=len(node_id)
N=Nj-Nr

V=0.3048*np.ones(Np)

q=np.ones(Np)
c1=np.pi/4
q=c1*diam*diam*V # m^3/s


nu=0.000001  #m^2/s

h=np.ones(Nj)
Re=np.zeros(Np)
epsD=np.zeros(Np)

epsD=rough/diam

print('initializing variables, time=',timer() - start,'s')

file read, time= 0.2197361 s


initializing variables, time= 0.07570830000000006 s


In [7]:
################################
#this must be repeated several times

In [8]:
n_iter=30
for jj in range(n_iter):

    print('\n iter',jj)

    ###############
    ## Reynolds  ##
    ###############
    start = timer()
    ###############
    Re=(4/(np.pi*nu))*abs(q)/diam;

    oo=np.ones(len(Re))

    f=np.zeros(Np)
    fp=np.zeros(Np)

    print('Calculate Re, time=',timer() - start,'s')

    start=timer()
    a=np.zeros((2,len(Re)))
    a1=np.where(Re<=2000,1,0)
    a3=np.where(Re>=4000,1,0)
    a2=np.ones(len(Re))-a1-a3
    a=f_lam(epsD[a1],Re[a1])*a1+f_trans(epsD,Re)*a2+f_turb(epsD,Re)*a3
    f=a[0]
    fp=a[1]
    print('f&fp- pure numpy way',timer()-start)



    #print('\n')



    ###############
    ## friction  ##
    ###############

    #print('turbulent f')
    #%timeit f_swamee(epsD,Re)
    #print('transition f')
    #%timeit f_dunlop(epsD,Re)
    #print('laminar f')
    #%timeit f_lam(epsD,Re)


    ###############
    start = timer()
    ###############
    a=f_tot(epsD,Re)
    f=a[0]
    fp=a[1]
    print('f&fp - jit -time=',timer() - start,'s')


    start=timer()
    a1=np.where(Re<=2000,True,False)
    a3=np.where(Re>=4000,True,False)
    a2=~(a1 | a3)
    a=np.zeros((2,len(Re)))
    a[:,a1]=f_lam(epsD[a1],Re[a1])
    a[:,a2]=f_trans(epsD[a2],Re[a2])
    a[:,a3]=f_turb(epsD[a3],Re[a3])
    f=a[0]
    fp=a[1]    
    print('f&fp - pure numpy way-2',timer()-start)


    #print('\n')


    ###############
    ## hL & g    ##
    ###############

    ###############
    start = timer()
    ###############

    #c1=0.0826268572006832
    c1=8/(np.power(np.pi,2)*9.81)
    r_turb=c1*len_/(diam*diam*diam*diam*diam)
    #r_turb=c1*len_/np.power(diam,5)
    hL=r_turb*f*q*abs(q)

    g=r_turb*abs(q)*(2*f+Re*fp)

    print('calculate h_L & g, time=',timer() - start,'s')



    #print('\n')

    ###############
    ##     A     ##
    ###############

    ###############
    start = timer()
    ###############

    nnn=(Nr+1)+(Nj+1)
    A=sparse.csr_matrix((nnn, nnn))

    a=1/g
    i=pipe_node[0]
    j=pipe_node[1]

    A=A+sparse.csr_matrix((-a, (i, j)), shape=(nnn, nnn))
    A=A+sparse.csr_matrix((-a, (j, i)), shape=(nnn, nnn))
    A=A+sparse.csr_matrix(( a, (i, i)), shape=(nnn, nnn))
    A=A+sparse.csr_matrix(( a, (j, j)), shape=(nnn, nnn))

    B=A
    A=sparse.csr_matrix((Nj+1, Nj+1))

    A=B[0:Nj,0:Nj]
    print('matrix A, time=',timer() - start,'s')


    #print('\n')


    ###############
    ##     F     ##
    ###############

    ###############
    start = timer()
    ###############

    F=np.zeros(nnn)


    b=-q+hL/g




    ###############
    #start = timer()
    ###############    

    
    #del i , j

    #F=np.bincount(i,weights=b,minlength=nnn)-np.bincount(j,weights=b,minlength=nnn)
    smoke1(F, np.asarray(b), np.asarray(i), np.asarray(j),Np)
    stor_ind_i=np.where(i>=Nj)[0]
    stor_ind_j=np.where(j>=Nj)[0]
    for kk in range(len(stor_ind_i)):
        k=stor_ind_i[kk]
        F[j[k]]+=rsrv_1[i[k]-Nj]/g[k]
    for kk in range(len(stor_ind_j)):
        k=stor_ind_j[kk]
        F[i[k]]+=rsrv_1[j[k]-Nj]/g[k]  
    
    #original slow code
    #for k in range(Np):
        #i=pipe_node[0][k]
        #j=pipe_node[1][k]
        #i1=i[k]
        #j1=j[k]
        #F[i1]=F[i1]+b[k]
        #F[j1]=F[j1]-b[k]
        #if i1>=Nj:
            #F[j1]=F[j1]+rsrv_[1][i1-Nj]/g[k];
        #elif j1>=Nj:
            #F[i1]=F[i1]+rsrv_[1][j1-Nj]/g[k];

    FF=F
    F=np.zeros(Nj)
    F[0:Nj]=FF[0:Nj]
    F=F-demand

    print('vecF, time=',timer() - start,'s')

    #print('\n')


    
    
    
    ###############
    ##   solve   ##
    ###############

    ###############
    start = timer()
    ###############

    h=sparse.linalg.spsolve(A,F)
    print('solve system, time=',timer() - start,'s')
    #%timeit sparse.linalg.spsolve(A,F)



    #print('\n')
    
    ###############
    ## update q  ##
    ###############

    ###############
    start = timer()
    ###############

    dq=np.zeros(Np);
    #dq=hL/g
    hh=np.append(h,rsrv_[1])  #it's easier to do this than handle indices
    
    dq=(hL-hh[i]+hh[j])/g;
    #original slow loop
    #for k in range(Np):
        #i1=i[k]
        #j1=j[k]
        #dq[k]=(hL[k]-hh[i[k]]+hh[j[k]])/g[k];
        #if i1<Nj and j1<Nj:
            #dq[k]=(hL[k]-h[i1]+h[j1])/g[k];
        #elif i1>=Nj:
            #dq[k]=(hL[k]-rsrv_[1][i1-Nj]+h[j1])/g[k]
        #elif j1>=Nj:
            #dq[k]=(hL[k]-h[i1]+rsrv_[1][j1-Nj])/g[k]
    
    q=q-dq

    print('updating q, time=',timer() - start,'s')
    print(q*1000)


 iter 0
Calculate Re, time= 0.00240700000000027 s
f&fp- pure numpy way 0.07146259999999982
f&fp - jit -time= 1.4843780000000004 s
f&fp - pure numpy way-2 0.049877099999999785
calculate h_L & g, time= 0.013177200000000333 s
matrix A, time= 0.05708529999999978 s
vecF, time= 0.18582490000000007 s
solve system, time= 0.2280413000000001 s
updating q, time= 0.006474100000000149 s
[-7.91351255  0.2232     -8.16315355 ...  0.073334    0.058981
  0.021212  ]

 iter 1
Calculate Re, time= 0.002638999999999392 s
f&fp- pure numpy way 0.07648860000000024
f&fp - jit -time= 0.0026946000000007686 s
f&fp - pure numpy way-2 0.04614269999999987
calculate h_L & g, time= 0.01719849999999923 s
matrix A, time= 0.06454260000000023 s
vecF, time= 0.005456300000000525 s
solve system, time= 0.23704280000000022 s
updating q, time= 0.00823380000000018 s
[-7.04739923  0.2232     -7.29704023 ...  0.073334    0.058981
  0.021212  ]

 iter 2
Calculate Re, time= 0.002684399999999698 s
f&fp- pure numpy way 0.066883699999

matrix A, time= 0.061860399999998705 s
vecF, time= 0.005210399999999282 s
solve system, time= 0.22921760000000013 s
updating q, time= 0.005443599999999549 s
[-7.18208725  0.2232     -7.43172825 ...  0.073334    0.058981
  0.021212  ]

 iter 19
Calculate Re, time= 0.0032713000000015313 s
f&fp- pure numpy way 0.07140889999999978
f&fp - jit -time= 0.004001900000000447 s
f&fp - pure numpy way-2 0.039243000000000805
calculate h_L & g, time= 0.01573720000000023 s
matrix A, time= 0.07033249999999924 s
vecF, time= 0.00551740000000045 s
solve system, time= 0.22203630000000096 s
updating q, time= 0.00539009999999962 s
[-7.18208725  0.2232     -7.43172825 ...  0.073334    0.058981
  0.021212  ]

 iter 20
Calculate Re, time= 0.01818070000000027 s
f&fp- pure numpy way 0.10869359999999872
f&fp - jit -time= 0.003007500000000718 s
f&fp - pure numpy way-2 0.0369761000000004
calculate h_L & g, time= 0.015754299999999333 s
matrix A, time= 0.06178740000000005 s
vecF, time= 0.005427700000000257 s
solve sys

In [9]:
q*1000
#expected result
#array([-7.18357863,  0.2232    , -7.43321963, ...,  0.073334  ,        0.058981  ,  0.021212  ])

array([-7.18208725,  0.2232    , -7.43172825, ...,  0.073334  ,
        0.058981  ,  0.021212  ])

In [10]:
np.savetxt(name_[name_n]+'\\q.csv', q, delimiter=",")
np.savetxt(name_[name_n]+'\\Re.csv', Re, delimiter=",")
np.savetxt(name_[name_n]+'\\epsD.csv', epsD, delimiter=",")

In [11]:
fp

array([-2.56645378e-08,  4.87237391e-06, -2.44444427e-08, ...,
       -4.26244640e-05, -6.58939309e-05, -5.09455720e-04])

In [12]:
a[1]

0.16874629226185403

In [13]:
f_lam(epsD[a1],Re[a1])

[array([0.08228382, 0.04778767, 0.08010687, ..., 0.05222993, 0.06494006,
        0.18056901]),
 array([-1.05791060e-04, -3.56822137e-05, -1.00267347e-04, ...,
        -4.26244640e-05, -6.58939309e-05, -5.09455720e-04])]

In [14]:
perm=reverse_cuthill_mckee(A)
perm
def permute_sparse_matrix(M, row_order=None, col_order=None):
    """
    Reorders the rows and/or columns in a scipy sparse matrix to the specified order.
    """
    if row_order is None and col_order is None:
        return M
    
    new_M = M
    if row_order is not None:
        I = sparse.eye(M.shape[0]).tocoo()
        I.row = I.row[row_order]
        new_M = I.dot(new_M)
    if col_order is not None:
        I = sparse.eye(M.shape[1]).tocoo()
        I.col = I.col[col_order]
        new_M = new_M.dot(I)
    return new_M
#%timeit permute_sparse_matrix(A,perm,perm)

In [15]:
#example assemble
Np=7   #number of pipes
Nj=5  #number of junctions (nodes)
Nr=1
pipe=[0,1,2,3,4,5,6]  #number of pipes are consecutive, k=0:Np
i=[0,1,2,3,3,3,0]     #number of start node for each pipe, 0<=i<Nj
j=[1,5,1,1,2,4,5]     #number of end node for each pipe, 0<=j<Nj
b=[1.3,0.5,1.5,2.7,1.5,2.2,3.1]   #value for each pipe, Np elements
node=[0,1,2,3,4,5]   #number of nodes are consecutive, 0:Nj
F=np.zeros(Nj+Nr+2)   #intializing F, size is equal to number of nodes

nn=Nj+Nr
for k in range(Np):   #iterate over pipes
    F[i[k]]=F[i[k]]+b[k]   #add value of b if pipe k is at the start of the node
    F[j[k]]=F[j[k]]-b[k]   #substract b if pipe k is at the end of the node

F

array([ 4.4, -5. ,  0. ,  6.4, -2.2, -3.6,  0. ,  0. ])